# Housing Prices Predictions

## 1. Data Exploration

In [ ]:
import sklearn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
np.random.seed(88)

In [ ]:
plt.rc('font', size=12)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=12)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)
plt.rc('figure', titlesize=16)
plt.rcParams['figure.figsize'] = [10, 6]

In [ ]:
housing = pd.read_csv("data/chap2/housing.csv")